The idea of this optimizer is to find the most optimal portfolio in terms of low correlation, high past returns, diverse country and branch distribtution 

toDO:
- add etf composition chart
- add etf top 10 most value stocks
- add anual return of ytd, past 5 years
- add weight to do portfolio analysis
- clean up code, still super messy

In [11]:
# save as etf_correlation.py and run: python etf_correlation.py
import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [12]:
# tickers for etfs
tickers = [
    "INRG.L", # Global Clean Energy
    "URTH", # MSCI World
    "IEUR", # MSCI Europe
    "EEM", # MSCI Emerging Markets
]

In [ ]:
# Load historical price data from CSV file
if not os.path.exists("etf_historical_data.csv"):
    raise FileNotFoundError("etf_historical_data.csv not found. Please provide the CSV file with ETF price data.")

# Read CSV with long format (Date, Ticker columns)
df = pd.read_csv("etf_historical_data.csv")
print(f"Loaded {len(df)} rows from etf_historical_data.csv")
print(f"Unique tickers: {df['Ticker'].unique()}")

# Convert Close to numeric, coerce errors to NaN
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

# Parse dates and normalize to remove timezone info
df['Date'] = pd.to_datetime(df['Date'], utc=True).dt.tz_localize(None).dt.normalize()

print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")

# Pivot to wide format: rows=dates, columns=tickers, values=Close prices
prices = df.pivot(index='Date', columns='Ticker', values='Close')

# Forward fill missing values (max 5 days) then drop any remaining NaNs
prices = prices.ffill(limit=5).dropna()

print(f"\nPivoted to {len(prices)} rows x {len(prices.columns)} columns")
print(f"Tickers in prices: {list(prices.columns)}")
print(f"Date range in prices: {prices.index.min()} to {prices.index.max()}")
print(f"\nFirst few rows:")
print(prices.head())

Loaded 5021 rows from etf_historical_data.csv
Unique tickers: ['INRG.L' 'URTH' 'IEUR' 'EEM']
Date range: 2020-12-09 00:00:00+00:00 to 2025-12-08 00:00:00-05:00


ValueError: Tz-aware datetime.datetime cannot be converted to datetime64 unless utc=True, at position 75

In [9]:
#calculate daily returns for all etfs for the past 5 years
returns = prices.pct_change().dropna()
returns = returns[-5*252:]  # last 5 years, approx. 252 trading days per year

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
# get stock holdings for the ETFs using yfinance data
